In [ ]:
import argparse
import numpy as np
import csv
import cv2
import time
import augmentation

In [ ]:
# base_path =  '/Users/sumitasok/Documents/Self-Driving Car/Behavioural Cloning/Training Data/'
base_path =  '/Users/sumitasok/Documents/Self-Driving Car/Behavioural Cloning/data/'

In [ ]:
lines = []
with open(base_path + 'driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

In [ ]:
images = []
measurements = []

for line in lines:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = base_path + 'IMG/' + filename
    # current_path = base_path + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)

In [5]:
# Data Augmentation
str_imgs, str_msr, agl_imgs, agl_msr = augmentation.split_straight_angle(images, measurements)
str_images, str_measurements = str_imgs, str_msr # augmentation.remove_excess_straigth_drive(str_imgs, str_msr, len(agl_imgs)/len(str_imgs))
agl_images, agl_measurements = augmentation.invert_images_and_measurements(agl_imgs, agl_msr)

In [6]:
# X_train = np.array(images)
# y_train = np.array(measurements)
X_train = np.array(str_images + agl_images)
y_train = np.array(str_measurements + agl_measurements)

print("straight images count: ", len(str_images), "measurements count: ", len(str_measurements), len(X_train))
print("angle images count: ", len(agl_images), "measurements count: ", len(agl_measurements), len(y_train))

straight images count:  9027 measurements count:  9027 9983
angle images count:  956 measurements count:  956 9983


In [7]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Dropout
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.pooling import MaxPooling2D
# from keras.layers.advanced_activations import ELU

Using TensorFlow backend.


In [8]:
model = Sequential()
# normaliastion of training data.
# https://keras.io/layers/convolutional/#cropping2d
model.add(Cropping2D(cropping=((64, 23),(0, 0)), input_shape=(160, 320, 3)))
# model.add(Lambda(lambda x: ((x/255.0)-0.5), input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: ((x/255.0)-0.5), input_shape=(100, 180, 3)))
model.add(Conv2D(6,5,5, activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(6,5,5, activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(4,3,3, activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(127))
# model.add(Dropout(0.5))
model.add(Dense(84))
# model.add(ELU)
model.add(Dense(24))
# model.add(ELU)
# model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=7)

/Users/sumitasok/miniconda3/envs/carnd-tf_keras_update/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
  import sys
/Users/sumitasok/miniconda3/envs/carnd-tf_keras_update/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
  if __name__ == '__main__':
/Users/sumitasok/miniconda3/envs/carnd-tf_keras_update/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()


Train on 7986 samples, validate on 1997 samples
Epoch 1/7
7986/7986 [==============================] - 303s - loss: 0.0097 - val_loss: 0.1523
Epoch 2/7
7986/7986 [==============================] - 228s - loss: 0.0058 - val_loss: 0.1530
Epoch 3/7
7986/7986 [==============================] - 255s - loss: 0.0056 - val_loss: 0.1460
Epoch 4/7
7986/7986 [==============================] - 251s - loss: 0.0053 - val_loss: 0.1499
Epoch 5/7
7986/7986 [==============================] - 267s - loss: 0.0053 - val_loss: 0.1461
Epoch 6/7
7986/7986 [==============================] - 255s - loss: 0.0051 - val_loss: 0.1481
Epoch 7/7
7986/7986 [==============================] - 227s - loss: 0.0050 - val_loss: 0.1471


In [9]:
timestamp = str(time.time()*1000000)
print("file identifier: ", timestamp)
# https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model
model_json = model.to_json()
with open("results/model-"+ timestamp +".json", "w") as json_file:
  json_file.write(model_json)
model.save('results/model-'+ timestamp +'.h5')
model.summary()
from keras.utils import plot_model
plot_model(model, to_file='results/model-'+timestamp+'.png')

file identifier:  1503738144154799.0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_1 (Cropping2D)    (None, 73, 320, 3)        0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 73, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 69, 316, 6)        456       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 34, 158, 6)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 154, 6)        906       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 77, 6)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13,

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.